In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from pydantic import BaseModel, TypeAdapter
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import requests
from datetime import datetime

In [3]:
OLLAMA_URL = "http://llama.lan:11434"

In [4]:
requests.get(OLLAMA_URL).text

'Ollama is running'

In [5]:
model = OpenAIModel(
    model_name='llama3.2',
    provider=OpenAIProvider(base_url=f'{OLLAMA_URL}/v1')
)

In [6]:
class OllamaModelDetails(BaseModel):
    parent_model: str
    format: str
    family: str
    families: list[str]
    parameter_size: str
    quantization_level: str

class OllamaModel(BaseModel):
    name: str
    model: str
    modified_at: datetime
    size: int
    digest: str
    details: OllamaModelDetails

In [7]:
import json

In [8]:
# use requests to get the model list
def get_model_list() -> list[OllamaModel]:
    try:
        response = requests.get(f"{OLLAMA_URL}/api/tags")
        response.raise_for_status()
        ollama_models_json = response.json().get("models", [])
        try:
            return TypeAdapter(list[OllamaModel]).validate_python(ollama_models_json)
        except ValueError as e:
            print(f"Error validating model list: {e}")
            return []
    except requests.RequestException as e:
        print(f"Error fetching model list: {e}")
        pass
    except ValueError as e:
        print(f"Error parsing model list: {e}")
        pass
    except Exception as e:
        print(f"Unexpected error: {e}")
        pass
    return []

In [9]:
ollama_models = get_model_list()

In [10]:
for index, ollama_model in enumerate(ollama_models):
    print(f"{index}: {ollama_model.name} ({ollama_model.model})")

0: llama3.2:latest (llama3.2:latest)
1: llama3.2:3b (llama3.2:3b)
2: nomic-embed-text:latest (nomic-embed-text:latest)
3: deepseek-coder:6.7b (deepseek-coder:6.7b)
4: deepseek-r1:14b (deepseek-r1:14b)
5: gemma3:12b (gemma3:12b)
6: gemma3:4b (gemma3:4b)


In [72]:
manager_prompt_1 = """
You wear dual hats as an executive assistant and research assistant, you job it to help break down complext tasks into actionable
insights or queries.

What are the steps you'd need to take or questions you'd need answered in order to resolve the question below.
**Ask questions that could be reasonably answered by the entity states of a sophisticated home assistant deployment.**
**Ask steps that could be reasonably taken by the devices of a sophisticated home assistant deployment.**

Repsond in a numbered list without any additional commentary.

""".strip()

In [79]:
manager_prompt_2 = """
You are a planning agent tasked with designing a simple directed graph of departments that will collaboratively answer a user's question.

Each department is a node. Each node:
- Has a name ** You don't need to include "Department" in the name ** 
- Has a responsibility or domain
- May depend on another department’s answer

When one department depends on another, define an edge:
- From the source node
- To the target node
** The source node is asking the target node a question. **

We'll construct the graph upside down, start with a PR department that compiles the information from
the departments and provides a concise answer to the user's direct question/prompt.

The PR department will ask the other departments for their input.
In turn some or all of those departments may need to ask other departments for their input.
The graph should be directed and acyclic.
The PR department will be the root node.

Output a YAML object with the following template:
```yaml
graph:
    nodes:
        - name: <name>
          responsibility: <responsibility>
        - name: <name>
          responsibility: <responsibility>
    edges:
        - from: <source_node_name>
          to: <target_node_name>
          question: <question>
```

Only include relevant departments. Be concise and logical.

Do not add explanations or commentary. Only return the YAML graph.

""".strip()

In [80]:
agent_1 = Agent(
    model=model,
    system_message=manager_prompt_1,
)
agent_2 = Agent(
    model=model,
    system_message=manager_prompt_2,
)

In [87]:
question = "Turn on all the christmas lights?"

considerations = agent_1.run_sync(f'{manager_prompt_1}\n\nQuestion: {question}')
result = agent_2.run_sync(f'{manager_prompt_2}\n\nConsiderations: {considerations.output}\n\nQuestion: Is today a good day to swim in the pool?')

In [88]:
print("Question:", question)
print("\n")
print("Considerations:")
print(considerations.output)

print("\n")
print(result.output)

Question: Turn on all the christmas lights?


Considerations:
1. Are Christmas lights currently turned off?
2. Is the presence of Christmas lights detected by the home assistant's lighting sensor?
3. Does the home assistant require power to control the lighting system?
4. Can the home assistant send a command to turn on the Christmas lights through its current setup?
5. Do Christmas lights have a default "on" state, or can they be set to "auto-on" upon detection of ambient light conditions?


```yaml
graph:
    nodes:
        
        - name: Pool Maintenance
          responsibility: Monitoring pool water quality and temperature
        
        - name: Ambiance Lighting
          responsibility: Setting lighting ambiance for pool area
        
        - name: Smart Home System
          responsibility: Controlling and managing smart home devices
        
        - name: Power Distribution
          responsibility: Distributing power to various smart home systems
        
        - na